In [103]:
import base64
import requests
import json
import os
import pandas as pd
from tqdm import tqdm

In [105]:
url = "http://192.168.0.199:5000/api/v1/detect-liveness"
models = ['fasnet', 'FTNet', 'default', 'rose_full_374']
images_path = '/Users/yoshlikmedia/Desktop/'
df = pd.DataFrame()
folders = ['AA3642573/', 'AA4469267/', 'AC2155899/']

In [98]:
images

['33492de2ea40413aac79acfcfd6540b0.jpeg',
 '4d58d4c77b6b40a882dfd16e70968c67.jpeg',
 '4d95848bfdbd499286a8757902a57999.jpeg',
 '850d04eaecce4111b8d9a7ee9544fc27.jpeg',
 'af3d2790e7eb461eb905f13938f432f2.jpeg',
 '87d59247cb4345dfa305a97c864a9330.jpeg',
 '5652c60b23274c739c4b12191dd0b62c.jpeg',
 '894f1cd5eceb49af833f93f9363a3848.jpeg',
 '8c74275e403e4ab19cd00f4c71250948.jpeg',
 '287be22ceb4b408eb8ba2ada0f553c1f.jpeg',
 '062369d0e4ef45878439c8980607325b.jpeg',
 '620f1a42f8994fec8c69bc472b376e9a.jpeg',
 '69fbbde6382b44abb567ad246958247c.jpeg']

In [99]:
def base64_encode(image_path):
    """
    Encode images Data URI - data:contenttype;base64,encoded_string
    :param image_path:
    :return:
    """
    with open(image_path, "rb") as image_file:
        encoded_string = base64.b64encode(image_file.read())
        encoded_string = encoded_string.decode("utf-8")
        encoded_string = "data:image/png;base64," + encoded_string
        return encoded_string


In [109]:
single_face_only = True
with_filter = True
open_eyes_only = True
color_image_only = False

for folder in folders:
    # get .jpg images
    images = [f for f in os.listdir(images_path + folder) if f.endswith('.jpg')]
    images = os.listdir(images_path + folder)
    print(images)
    for image in tqdm(images):
        model_ans = {}
        image_path = images_path + folder + image
        encoded_string = base64_encode(image_path)
        for model in tqdm(models):
            payload = json.dumps({
                "model": "{}".format(model),
                "single_face_only": single_face_only,
                "with_filter": with_filter,
                "open_eyes_only": open_eyes_only,
                "color_image_only": color_image_only,
                "image_b64": "{}".format(encoded_string)
            })
            headers = {
                'Content-Type': 'application/json'
            }
            response = requests.request("POST", url, headers=headers, data=payload)
            try:
                result = response.json()['liveness_result']       
                model_ans[model] = result['is_real']
            except Exception as ex:
                try:
                    model_ans[model] = response.json()['detail']
                except Exception as ex:
                    print(ex)
        try: 
            data = {
                'image': image,
                'fasnet': model_ans['fasnet'],
                'FTNet': model_ans['FTNet'],
                'default': model_ans['default'],
                'rose_full_374': model_ans['rose_full_374'],
            }    
            
            df = df.append(data, ignore_index=True)
        except Exception as ex:
            print(ex)

        
print('DONE!')

['cb120c14e3494f7ba488ac1034c1ec92.jpeg', 'efbdb4a220144947bb6fee378f930479.jpeg', '.DS_Store', '8f7337232f6e45fcb6b55d9f00cb69d5.jpeg', 'b8d10b1d2cff4ac0ba770e169878ab48.jpeg', '3e20ab0dbb8244c68531c52dc17987ce.jpeg', 'e8f7ad09e9dc4b01a1f35ecf555c2c94.jpeg', '65ceda25b9314bff8f421d9032bb59f1.jpeg', '75d3fc9c823f4130b38a6a0ebf59d0a6.jpeg', '660bdd0648eb4580b9831f37c3677ca8.jpeg', 'ef35dd5f946f42b489102e10203f4328.jpeg', '190d69bfccfb4623ae5ffccb20c021a6.jpeg', 'AA3642573.xlsx', '0420cdf02b9a498aa1aaa51df32ce114.jpeg', 'c7ec3c80ee73409883b68e29810dcde1.jpeg', 'fb6a86055f0748c3830a4effc9c40025.jpeg', '2496e2eec9824878897b98b9985eb91e.jpeg', '0beb577823644cb0ad8dad753c642edd.jpeg', '7218ccebf86a4a929e331ff46946d409.jpeg', 'b43400bfe6b649b1ba5c7a4fb58dde8a.jpeg', '47232a78dce9417083d13f062a53d657.jpeg', 'ea29c25af69541bdaa96d108d67b9025.jpeg', '812763a84e1b49be99382f90e50cd223.jpeg', 'b54d9138c35b4d188c3308bace3dc54c.jpeg', 'f9ef1862049f46bcabfcd641ad579fe8.jpeg', 'd10ed88c734346478d0fc3ca

In [101]:
df

,image,fasnet,FTNet,default,rose_full_374
0,33492de2ea40413aac79acfcfd6540b0.jpeg,True,True,True,True
1,4d58d4c77b6b40a882dfd16e70968c67.jpeg,False,False,True,True
2,4d95848bfdbd499286a8757902a57999.jpeg,False,False,True,True
3,850d04eaecce4111b8d9a7ee9544fc27.jpeg,False,False,Multiple faces detected in the image,Multiple faces detected in the image
4,af3d2790e7eb461eb905f13938f432f2.jpeg,False,False,True,True
5,87d59247cb4345dfa305a97c864a9330.jpeg,False,False,Multiple faces detected in the image,Multiple faces detected in the image
6,5652c60b23274c739c4b12191dd0b62c.jpeg,False,False,False,True
7,894f1cd5eceb49af833f93f9363a3848.jpeg,Bad or blurry image,Bad or blurry image,Bad or blurry image,Bad or blurry image
8,8c74275e403e4ab19cd00f4c71250948.jpeg,Bad or blurry image,Bad or blurry image,Bad or blurry image,Bad or blurry image
9,287be22ceb4b408eb8ba2ada0f553c1f.jpeg,Bad or blurry image,Bad or blurry image,Bad or blurry image,Bad or blurry image


In [102]:
# save to csv
df.to_excel(f'{images_path}/AC2155899.xlsx')